In [1]:
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import *
import numpy as np
import sys

In [2]:
import taurex.log
taurex.log.disableLogging()

In [3]:
from taurex.cache import OpacityCache,CIACache
OpacityCache().clear_cache()
OpacityCache().set_opacity_path("../atmosphere/xsecs")
CIACache().set_cia_path("../atmosphere/cia/hitran")


In [6]:
from taurex.temperature import Guillot2010
guillot = Guillot2010(T_irr=1200.0)

In [7]:
from taurex.planet import Planet
planet = Planet(planet_radius=1.7420,planet_mass=1.170)  #in jupiter masses

#parameters retrieved from https://ui.adsabs.harvard.edu/abs/2024AJ....168..231S/abstract

In [8]:
from taurex.stellar import BlackbodyStar

star = BlackbodyStar(temperature=6628,radius=1.461)

Numba not installed, using numpy instead


In [9]:
from taurex.chemistry import TaurexChemistry
from taurex.chemistry import ConstantGas
chemistry = TaurexChemistry(fill_gases=['H2','He'],ratio=0.172)
# H2O, CH4, CO2 and CO randomised within [10^{-8},10^{-2}].
rng = np.random.default_rng(seed=100)
chemistry.addGas(ConstantGas('H20',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CH4',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CO2',mix_ratio=rng.uniform(1e-8,1e-2)))
chemistry.addGas(ConstantGas('CO',mix_ratio=rng.uniform(1e-8,1e-2)))

In [10]:
from taurex.model import TransmissionModel
from taurex.contributions import AbsorptionContribution
from taurex.contributions import CIAContribution
from taurex.contributions import RayleighContribution

tm = TransmissionModel(planet=planet,
                       temperature_profile=guillot,
                       chemistry=chemistry,
                       star=star,
                       atm_min_pressure=1e-0,
                       atm_max_pressure=1e6,
                       nlayers=30)
tm.add_contribution(AbsorptionContribution())
tm.add_contribution(CIAContribution(cia_pairs=['H2-H2','H2-He']))
tm.add_contribution(RayleighContribution())
tm.build()
res = tm.model()
res

(array([  199.99326855,   200.00660143,   200.01993521, ...,
        33328.88933329, 33331.11125925, 33333.33333333], shape=(76744,)),
 array([0.0160977 , 0.01613024, 0.0159089 , ..., 0.01583692, 0.01583695,
        0.01583698], shape=(76744,)),
 array([[0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         1.08710276e-010, 1.08710276e-010, 1.08710276e-010],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         1.10576658e-264, 9.28936531e-265, 7.80343340e-265],
        [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
         8.01701937e-167, 7.18002676e-167, 6.43020330e-167],
        ...,
        [9.33559432e-001, 8.12833966e-001, 9.94108024e-001, ...,
         9.96181405e-001, 9.96180296e-001, 9.96179186e-001],
        [9.71893807e-001, 8.59807327e-001, 9.96763477e-001, ...,
         9.97876359e-001, 9.97875741e-001, 9.97875123e-001],
        [9.89408347e-001, 9.28885909e-001, 9.98575574e-001, ...,
         9.99081512e-001, 9.99081245e-001, 9.9